# TYTX Base - Basic Types

This notebook demonstrates the basic type encoding/decoding with TYTX Base.

## Setup

In [ ]:
from datetime import date, datetime, time
from decimal import Decimal

from genro_tytx import to_typed_text, from_text, to_typed_json, from_json

## Encoding Basic Types

TYTX Base encodes non-native JSON types with a `::suffix` marker.

In [ ]:
# Decimal - exact numeric precision
data = {"price": Decimal("100.50")}
encoded = to_typed_text(data)
print(f"Decimal: {encoded}")

In [ ]:
# Date
data = {"date": date(2025, 1, 15)}
encoded = to_typed_text(data)
print(f"Date: {encoded}")

In [ ]:
# DateTime
data = {"timestamp": datetime(2025, 1, 15, 10, 30, 0)}
encoded = to_typed_text(data)
print(f"DateTime: {encoded}")

In [ ]:
# Time
data = {"time": time(10, 30, 0)}
encoded = to_typed_text(data)
print(f"Time: {encoded}")

## Native Types Pass Through

Native JSON types (int, float, bool, str, None) are not encoded with suffixes.

In [ ]:
# Native types - no ::JS suffix
data = {"name": "Widget", "count": 42, "active": True, "price": 99.99}
encoded = to_typed_text(data)
print(f"Native types: {encoded}")
print(f"No ::JS suffix: {'::JS' not in encoded}")

## Mixed Types

In [ ]:
# Mixed native and typed values
data = {
    "name": "Widget",           # native str
    "count": 42,                 # native int
    "price": Decimal("99.99"),  # typed Decimal
    "date": date(2025, 1, 15),  # typed date
}
encoded = to_typed_text(data)
print(f"Mixed: {encoded}")
print(f"Has ::JS suffix: {'::JS' in encoded}")

## Decoding

In [ ]:
# Decode typed JSON
encoded = '{"price": "100.50::N", "date": "2025-01-15::D"}::JS'
decoded = from_text(encoded)
print(f"Decoded: {decoded}")
print(f"Price type: {type(decoded['price'])}")
print(f"Date type: {type(decoded['date'])}")

## Roundtrip

In [ ]:
# Perfect roundtrip
original = {
    "price": Decimal("99.99"),
    "date": date(2025, 1, 15),
    "time": time(10, 30, 0),
}

encoded = to_typed_text(original)
decoded = from_text(encoded)

print(f"Original: {original}")
print(f"Encoded:  {encoded}")
print(f"Decoded:  {decoded}")
print(f"Equal: {decoded == original}")

## Text vs JSON Format

- `to_typed_text` / `from_text`: Uses `::JS` suffix only
- `to_typed_json` / `from_json`: Uses `TYTX://` prefix + `::JS` suffix

In [ ]:
data = {"price": Decimal("100.50")}

# Text format
text_encoded = to_typed_text(data)
print(f"Text format:  {text_encoded}")

# JSON format
json_encoded = to_typed_json(data)
print(f"JSON format:  {json_encoded}")

In [ ]:
# from_json accepts both formats
result1 = from_json('{"price": "100.50::N"}::JS')  # without prefix
result2 = from_json('TYTX://{"price": "100.50::N"}::JS')  # with prefix

print(f"Without prefix: {result1}")
print(f"With prefix:    {result2}")
print(f"Equal: {result1 == result2}")

## Type Codes Summary

| Code | Type | Example |
|------|------|---------|
| `N` | Decimal | `"100.50::N"` |
| `D` | date | `"2025-01-15::D"` |
| `DHZ` | datetime | `"2025-01-15T10:30:00Z::DHZ"` |
| `H` | time | `"10:30:00::H"` |